In [ ]:
9  ## check current dir
!dir

In [ ]:
## del database using windows, demonstrate  db locking
!del library_copy.db

### open and get db connection

In [ ]:
import sqlite3

#Create Connection object called `connection` that opens library_copy.db database under the folder resources
connection=sqlite3.connect('library_copy.db')

#Do something with the connection


In [ ]:
#Close the connection, otherwise it is locked
connection.close()

# Data Definition Language (DDL)

### Drop Tables

In [ ]:

import sqlite3
## using try and finally to avoid locking up the database

try:
#Create connection objects called `con` that opens library_copy.db database under the folder resources
    con=sqlite3.connect('library_copy.db')

    #Do something with the con
    sql_str = "DROP TABLE IF EXISTS Book"
    con.execute(sql_str) ## drop table when table doesn't exists
## Drop the Borrower, Loan, Publisher tables if they exsit

finally:
#Close the con
    con.close()

### Create tables

In [ ]:
import sqlite3

#Create con object called `con` that opens library_copy.db database under the folder resources
con=sqlite3.connect('library_copy.db')

## Create Book table
## Copy from DBBrowser and edit 
sql_str = "\
CREATE TABLE IF NOT EXISTS`Book` (\
	`ID`	INTEGER NOT NULL,\
	`Title`	TEXT NOT NULL,\
	`PublisherID`	INTEGER,\
	`Damaged`	INTEGER NOT NULL,\
	FOREIGN KEY(`PublisherID`) REFERENCES `Publisher`(`ID`),\
	PRIMARY KEY(`ID`)\
);\
"
con.execute(sql_str)

sql = """ CREATE TABLE IF NOT EXISTS `Publisher` (
	`ID`	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,
	`Name`	TEXT NOT NULL
);"""

con.execute(sql)
con.close()

### Exercise 0
- Create the Borrower, Loan and Publisher Table

In [ ]:
import sqlite3

#Create con object called `con` that opens library_copy.db database under the folder resources
con=sqlite3.connect('library_copy.db')


In [ ]:

# Book Table
con.execute(" \
    CREATE TABLE IF NOT EXISTS `Book` ( \
	`ID`	INTEGER NOT NULL, \
	`Title`	TEXT NOT NULL, \
	`PublisherID`	INTEGER, \
	`Damaged`	INTEGER NOT NULL, \
	FOREIGN KEY(`PublisherID`) REFERENCES `Publisher`(`ID`), \
	PRIMARY KEY(`ID`) \
    )"    
)   

In [ ]:
# Borrower Table

con.execute("\
CREATE TABLE  IF NOT EXISTS `Borrower` (\
	`ID`	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,\
	`FirstName`	TEXT NOT NULL,\
	`Surname`	TEXT NOT NULL,\
	`Contact`	TEXT NOT NULL\
)"
)


In [ ]:
## Loan Table
con.execute("PRAGMA foreign_keys = ON")
con.execute("\
CREATE TABLE  IF NOT EXISTS `Loan` (\
	`ID`	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,\
	`BorrowerID`	INTEGER NOT NULL,\
	`BookID`	INTEGER NOT NULL,\
	`Date Borrowed`	TEXT,\
	FOREIGN KEY(`BookID`) REFERENCES `Book`(`ID`),\
	FOREIGN KEY(`BorrowerID`) REFERENCES `Borrower`(`ID`)\
)"
)


In [ ]:
## Publisher Table
con.execute("\
CREATE TABLE  IF NOT EXISTS `Publisher` (\
	`ID`	INTEGER NOT NULL PRIMARY KEY AUTOINCREMENT,\
	`Name`	TEXT NOT NULL\
)"
)


In [ ]:
con.close() 

____ 
# Data Manipulation Language (DML) - CRUD operations


In [ ]:
## INSERT STATEMENT,
## NOT NULL, UNIQUE, CONSTRAINTS
## NULL <> ""
import sqlite3
try:
    con = sqlite3.connect("library_copy.db")
    con.execute("INSERT INTO Borrower(FirstName, Surname, Contact) VALUES('AlexC', 'Ong', 98765432)")
    con.execute("INSERT INTO Borrower(FirstName, Surname, Contact) VALUES('VijayC', 'Singh', 91919191)")
    con.commit()
except:
    #the following line is used to UNDO all previous CRUD operations
    con.rollback() # Clear buffer

try:
    #con.execute("DELETE FROM Borrower")
    con.execute("INSERT INTO Borrower(FirstName, Surname, Contact) VALUES('', 'Ong', 98765432)") ## CHECK(length(contact) = 8)
    con.execute("INSERT INTO Borrower VALUES(199,'VijayD', 'Singh', 91919191)") ## custom ID is used to override the Auto Increment ID 
    con.commit() ## Acquire lock, Write changes from buffer to database
except:
    #the following line is used to UNDO all previous CRUD operations
    con.rollback() # Clear buffer

finally:
    con.close()

In [ ]:

import sqlite3
con = sqlite3.connect("library_copy.db")

## Using f-string in SQL str
#### NEVER DO THIS !!!

id = 999091
f_name = "Joe"
s_name = "Chan"
contact = "12345678"
sql_str = f"INSERT INTO Borrower VALUES({id},'{f_name}','{s_name}','{contact}')"
print(sql_str)
con.execute(sql_str)
con.commit()


### SQL Injection
- SQL Code is inserted as user input and executed on the database server


***Parameter Substituition***


Parameter substitution is used to execute SQL statements to safely include data that is provided by the user. To do this, we use the question-mark character ? as a placeholder in the SQL for any data that is provided by the user. We then provide a second argument to execute() that is a tuple or list of values that will replace the placeholders. This ensures that the provided values are escaped properly and cannot be misinterpreted as SQL.

Parameter substitution follows the same order in which the placeholders appear in the SQL. 
```
connection.execute("INSERT INTO Borrower(FirstName, Surname,Contact) 
                    VALUES(?, ?, ?)", (first_name, sur_name, contact) )
```




### Exercise 1
- Prompt user for first_name, sur_name and contact
- insert into the Borrower Table

In [ ]:
## Code


### importing data from files using INSERT
- Order of Table Insertion
- Referential Integrity
    - Ensures that all foreign keys are referencing a valid key in a table
    - Implement by using Foreign Key Constraint (need to be explcitly set in sqlite3 using PRAGMA commands)

In [ ]:
### INSERT Borrowers
import sqlite3
import csv

con = sqlite3.connect("library_copy.db")
con.execute("PRAGMA foreign_keys = ON")
con.execute("DELETE FROM Borrower")
csv_reader = csv.reader(open("Borrower.csv"))
data  = list(csv_reader)[1:]
for row in data:
    con.execute("INSERT INTO Borrower VALUES(?,?,?,?)", row)
con.commit()
con.close()


### Exercise 2 Insert data into the 4 tables

____
### UPDATE
- All the contact numbers of borrowers in  the Borrower table must add the international dialling pre-fix of "+65". Write down the SQL statement to update the Borrower table.

In [ ]:
import sqlite3
import csv
try:
    con = sqlite3.connect("library_copy.db")
    cur = con.cursor()
    cur.execute("PRAGMA ignore_check_constraints = ON;") ## if constraint for contact is enforced
    cur.execute("UPDATE Borrower SET Contact = '+65' || Contact ")
    print( f"{cur.rowcount} rows updated")
    con.commit()
    cur.execute("PRAGMA ignore_check_constraints = OFF;")
finally:
    con.close()


### Exercise 3
- All the books borrowed by Sarah (BorrowerID 2) should have been borrowed by Kumara (BorrowerID 3).

In [ ]:
## Code here


___ 
### SELECT STATEMENT


The SELECT command is used to select data from the database. When you run a SELECT command in DB Browser, the selected rows are usually displayed in a table.

In Python, however, you must access the selected rows using a cursor object that is returned by the execute() method. This cursor object can go through the selected rows, one by one, using a for-in loop. Each iteration returns a tuple of the columns in the current row. 

In [ ]:
import sqlite3

con = sqlite3.connect("library.db")
cursor = con.execute("SELECT ID, Title FROM Book")
rows = cursor.fetchall()
#rows = list(cursor)
con.close()
print(rows)
for row in rows:
    print(type(row), row[1])    # Title is second item in the tuple



Alternatively, we can configure the SQLite connection so that each row is retrieved as a dict mapping column names to values instead. 

To do this, we set the connection object's row_factory attribute to the built-in sqlite3.Row class. This lets us change the ordering of columns in our SELECT statements without having to modify the code for extracting individual column values.

In [ ]:
import sqlite3

con = sqlite3.connect("library.db")
con.row_factory = sqlite3.Row
cursor = con.execute("SELECT ID, Title FROM Book")
for row in cursor:
    print(type(row),row["Title"])    # row is now a dictionary
con.close()


### Exercise 4
- Retreive and print all the Borrower's name 
- Prompt the user which borrower to delete

### Exercise 5
- Retreive and print all the Borrower's name  and the Books that they borrow
